In [ ]:
# Importing

from langchain_core.embeddings import Embeddings
from infinispan_vector import Infinispan

In [ ]:
# Create a fake embedding class which generates 3 dimension vector

from typing import List
from matplotlib import colors
class RGBEmbeddings(Embeddings):
    """Fake embeddings which returns rgb vector given a matplotlib colorname
    """

    def __init__(self, dimensionality: int = 3) -> None:
        self.known_texts: List[str] = []
        self.dimensionality = dimensionality

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Return consistent embeddings for each text seen so far."""
        return [self.embed_query(text) for text in texts]


    def embed_query(self, text: str) -> List[float]:
        """Convert str to a json list of float and return it."""
        return list(colors.to_rgb(text))


In [ ]:
# Creating a langchain_core.VectorStore
# the default store name `embeddingvectors` will be used

infinispan = Infinispan.from_texts(embedding=RGBEmbeddings(),
                                 configuration={"lambda.content": lambda item: item["color"]})

In [ ]:
# Creating an Infinispan cache to store vectors

cache_def = '''
{
  "distributed-cache": {
    "owners": "2",
    "mode": "SYNC",
    "statistics": true,
    "encoding": {
      "media-type": "application/x-protostream"
    },
    "indexing": {
      "enabled": true,
      "storage": "filesystem",
      "startup-mode": "AUTO",
      "indexing-mode": "AUTO",
      "indexed-entities": [
        "vector"
      ]
    }
  }
}
'''
infinispan.req_cache_post("embeddingvectors",cache_def)

In [ ]:
data = '''
/**
 * @Indexed
 */
message vector {
/**
 * @Vector(dimension=3)
 */
repeated float floatVector = 1;
optional string texture = 2;
optional string color = 3;
}
'''
output = infinispan.req_schema_post("vector.proto",data)
print(output.text)
assert output.status_code == 200


In [ ]:
# Adding some data
texts =    [{"_key": 1, "_type": "vector", "texture" : "matt", "color" : "red"},
                {"_key": 2, "_type": "vector", "texture": "glossy", "color": "green"},
                {"_key": 3, "_type": "vector", "texture": "silk", "color": "blue"},
                {"_key": 4, "_type": "vector", "texture": "matt", "color": "black"},
                {"_key": 5, "_type": "vector", "texture": "raw", "color": "white"},
                                 ]
infinispan.add_texts(texts)

In [ ]:
# Running search

infinispan.similarity_search("orange",2)

In [ ]:
infinispan.similarity_search("purple",2)

In [ ]:
infinispan.similarity_search("lime",2)

In [ ]:
infinispan.similarity_search("snow",2)

In [ ]:
# Clean up
infinispan.req_cache_delete("embeddingvectors")
infinispan.req_schema_delete("vector.proto")